In [32]:
"""Google Trends data source collector"""

import time

import pandas as pd
from pytrends.request import TrendReq

BASE_URL = 'https://trends.google.com/trends/explore?date=all&q=%s'
UNIT = 'frekvence vyhledávání'

# Datasets to fetch along with their metadata
datasets = {
    '/m/0dfm1w': {
        'id': 'mortgage_calculator',
        'name': 'Hypoteční kalkulačka - Téma',
        'description': 'Mortgage calculator - Topic'
    }
}

# Regions to fetch the datasets for
# Google Trends codes along with DB codes
europe = {
    'AT': 'aut',
    'BE': 'bel',
    'BG': 'bgr',
    'HR': 'hrv',
    'CY': 'cyp',
    'CZ': 'cze',
    'DK': 'dnk',
    'EE': 'est',
    'FI': 'fin',
    'FR': 'fra',
    'DE': 'deu',
    'GR': 'grc',
    'HU': 'hun',
    'IE': 'irl',
    'IT': 'ita',
    'LV': 'lva',
    'LT': 'ltu',
    'LU': 'lux',
    'MT': 'mlt',
    'NL': 'nld',
    'PL': 'pol',
    'PT': 'prt',
    'RO': 'rou',
    'SK': 'svk',
    'SI': 'svn',
    'ES': 'esp',
    'SE': 'swe',
}

other_regions = {
    '': 'wld', # Empty region parameter means whole world
    'NO': 'nor',
    'GB': 'gbr'
}

def fetch(term, region, timeframe='all'):
    """Fetch data for a term, return yearly mean values"""

    pytrends = TrendReq(hl='en-US', tz=0)

    # Fetch monthly data
    pytrends.build_payload([term], timeframe=timeframe, geo=region)
    data = pytrends.interest_over_time()

    if data.empty:
        return data

    print(data)
    if 'isPartial' in data.columns:
        data.drop(columns='isPartial', inplace=True)

    # Convert to yearly values as mean of months
    data.reset_index(level=0, inplace=True)
    data.date = data.date.astype('string')
    data = pd.concat([
        data.date.str[0:4],
        data[term]
        ], axis=1)
    data.rename(columns = {term: 'value'}, inplace=True)
    data.set_index('date', inplace=True)
    return data.groupby(['date'])['value'].mean()

def collect():
    """Collect data from the data source"""

    for term, props in datasets.items():
        print('  - ' + props['name'])

        # Process European Union countries
        europe_data = pd.Series(dtype='float64')
        for country, region_id in europe.items():
            print('    - ' + country)
            term_fetch = str(term)
            data = fetch(term=term_fetch, region=country)
            if not data.empty:
                europe_data = pd.concat([europe_data, data])

            # Avoid rate limiting
            time.sleep(1)

        # Create a European mean
        europe_data = europe_data.groupby(level=0).mean()

    # Collect data for other regions
        for region, region_id in other_regions.items():
            print('    - ' + region)
            data = fetch(term=term, region=region)

            # Avoid rate limiting
            time.sleep(1)


In [33]:
collect()

  - Hypoteční kalkulačka - Téma
    - AT
            /m/0dfm1w  isPartial
date                            
2004-01-01         58      False
2004-02-01         60      False
2004-03-01          0      False
2004-04-01         61      False
2004-05-01         52      False
...               ...        ...
2021-12-01         20      False
2022-01-01         33      False
2022-02-01         15      False
2022-03-01         20      False
2022-04-01         15       True

[220 rows x 2 columns]
    - BE
            /m/0dfm1w  isPartial
date                            
2004-01-01         78      False
2004-02-01         27      False
2004-03-01          0      False
2004-04-01         27      False
2004-05-01          0      False
...               ...        ...
2021-12-01         14      False
2022-01-01         10      False
2022-02-01         10      False
2022-03-01         15      False
2022-04-01          9       True

[220 rows x 2 columns]
    - BG
            /m/0dfm1w  isPartial
da

In [9]:
for term, props in datasets.items():
    for country, region_id in europe.items():
        pytrends.build_payload([term], timeframe='all', geo=country)
        data = pytrends.interest_over_time().drop(columns="isPartial")

        print(data)

            /m/01cnz
date                
2004-01-01         0
2004-02-01        59
2004-03-01        18
2004-04-01        30
2004-05-01        26
...              ...
2021-12-01        27
2022-01-01        39
2022-02-01        23
2022-03-01        33
2022-04-01        19

[220 rows x 1 columns]
            /m/01cnz
date                
2004-01-01        98
2004-02-01        76
2004-03-01        94
2004-04-01        80
2004-05-01        97
...              ...
2021-12-01        58
2022-01-01        66
2022-02-01        64
2022-03-01        71
2022-04-01        65

[220 rows x 1 columns]
            /m/0fw7r
date                
2004-01-01        32
2004-02-01        44
2004-03-01        40
2004-04-01       100
2004-05-01        10
...              ...
2021-12-01        36
2022-01-01        45
2022-02-01        57
2022-03-01        52
2022-04-01        43

[220 rows x 1 columns]
            /m/0fw7r
date                
2004-01-01        69
2004-02-01        65
2004-03-01        54
2004

In [17]:
import pandas as pd

# Wages
data = pd.read_csv('https://www.czso.cz/documents/62353418/143522174/110080-21data052421.csv')

# Extract data
data_men = data[(data['POHLAVI_kod'] == 1)
    & (data['uzemi_cis'] == 97)
    & (data['SPKVANTIL_cis'] == 7636)][['rok', 'hodnota']].set_index(['rok']).sort_index()

data_men = pd.Series(data=data_men['hodnota'], index=data_men.index)

data_men.name = 'wages_men'

data_women = data[(data['POHLAVI_kod'] == 2)
    & (data['uzemi_cis'] == 97)
    & (data['SPKVANTIL_cis'] == 7636)][['rok', 'hodnota']].set_index(['rok']).sort_index()

data_women = pd.Series(data=data_women['hodnota'], index=data_women.index)

data_women.name = 'wages_women'

# Save data
print('  - Wages men')
print(data_men)

print('  - Wages women')
print(data_women)

  - Wages men
rok
2011    23533
2012    23652
2013    23955
2014    24670
2015    25688
2016    26973
2017    29006
2018    31433
2019    33763
2020    35198
Name: wages_men, dtype: int64
  - Wages women
rok
2011    19731
2012    20042
2013    20271
2014    20660
2015    21461
2016    22651
2017    24477
2018    26678
2019    28795
2020    30843
Name: wages_women, dtype: int64
